<a href="https://colab.research.google.com/github/AnaBelenCarbajal/Thesis/blob/main/shapesTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osculari

In [64]:
# importing required packages
import osculari

import numpy as np
from matplotlib import pyplot as plt
import torch
import os

In [ ]:
!unzip images180.zip -d my_data

In [53]:
from torchvision.datasets import ImageFolder

In [66]:
labels = os.listdir("my_data/images180/images180")

In [67]:
labels[7]

'elephant'

In [48]:
!ls my_data/images180/images180

bird	   cat	crocodile  dog	     duck      fish	   hen	  leopard  rabbit  spider
butterfly  cow	deer	   dolphine  elephant  flyingbird  horse  monkey   rat	   tortoise


In [55]:
a = ImageFolder("my_data/images180/images180")

In [63]:
a[0]

(<PIL.Image.Image image mode=RGB size=180x180>, 0)